###### Il progetto è visibile su github al seguente indirizzo: https://github.com/albianto97/bigdataFile.

Il dataset analizzato (https://www.kaggle.com/datasets/shuhengmo/uber-nyc-forhire-vehicles-trip-data-2021), contiene informazioni sui viaggi fatti da varie licenze "taxi" nella città di New York.
La grandezza totale del dataset è di 5 GB e contiene molte informazioni sui luoghi di partenza, arrivo, prezzo e varie tempistiche come l'orario e giorno di partenza e quelli d'arrivo.

In [1]:
%%configure -f
{"executorMemory":"6G", "numExecutors":2, "executorCores":3, "conf":
{"spark.dynamicAllocation.enabled": "false"}}


Attach to a cluster to execute a cell.

In [1]:
val bucketname = "unibo-bd-antonelli2023"
val path_ml_tripdata =
"s3a://"+bucketname+"/datasets/fhvhv_tripdata_2021-01.parquet"
val path_ml_tripdata2 =
"s3a://"+bucketname+"/datasets/fhvhv_tripdata_2021-02.parquet"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1686342766862_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd-antonelli2023
path_ml_tripdata: String = s3a://unibo-bd-antonelli2023/datasets/fhvhv_tripdata_2021-01.parquet
path_ml_tripdata2: String = s3a://unibo-bd-antonelli2023/datasets/fhvhv_tripdata_2021-02.parquet
res2: String = application_1686342766862_0001
res4: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1686342766862_0001/


I dati sono presenti nel formato Parquet; dopo essere stati importati, tutti i job sono stati fatti dopo una successiva conversione in RDD. I dati sono raggruppati mensilmente, per un totale di 12 file Parquet. 

Importazione dei dati in formato Parquet.

In [2]:
import spark.implicits._
val parquetFileDF = spark.read.parquet(path_ml_tripdata)
val parquetFileDF2 = spark.read.parquet(path_ml_tripdata2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import spark.implicits._
parquetFileDF: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]
parquetFileDF2: org.apache.spark.sql.DataFrame = [hvfhs_license_num: string, dispatching_base_num: string ... 22 more fields]


# Analisi dei campi del dataset
* licenseClass: tipo stringa che rappresenta la licenza de taxi.
	* HV0002: Juno
    * HV0003: Uber
    * HV0004: Via
    * HV0005: Lyft
* license: tipo Stringa che rappresenta il numero del taxi.
* request: tipo Timestamp che rappresenta la data/ora della richiesta del viaggio.
* pickup: tipo Timestamp  che rappresenta la data/ora dell'inizio del viaggio.
* dropoff: tipo Timestamp che rappresenta la data/ora della fine del viaggio.
* distance: tipo Double che rappresenta la distanza totale in miglia percorsa dal taxi per questo viaggio.
* startloc: tipo Long che rappresenta la zona di partenza del viaggio.
* endloc: tipo Long che rappresenta la zona di arrivo del viaggio.
* fare: tipo Double che rappresenta il prezzo del viaggio.

In [3]:
import java.sql.Timestamp
case class TaxiTrip(
  licenseClass:String,
  license:String,  
  request:Timestamp,
  pickup:Timestamp,
  dropoff:Timestamp,
  distance:Double,
  startloc:Long,
  endloc:Long,
  //time:Long,
  fare:Double,
)

object TaxiTrip {
    def extract(row:org.apache.spark.sql.Row) = {
        val licenseClass = row.getString(0)
        val license = row.getString(1)
        val request = row.getTimestamp(4)
        val pickup = row.getTimestamp(5)
        val dropoff = row.getTimestamp(6)
        val distance = row.getDouble(9)
        val startloc = row.getLong(7)
        val endloc = row.getLong(8)
        //val time = row.getLong(10)
        val fare = row.getDouble(11)
        
        new TaxiTrip(licenseClass,license,request,pickup,dropoff,distance,startloc,endloc,fare)
        

        //new TaxiTrip(license,request,pickup,dropoff,startloc,endloc,distance,time,fare)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.sql.Timestamp
defined class TaxiTrip
defined object TaxiTrip
Companions must be defined together; you may wish to use :paste mode for this.


La seguente funzione serve per convertire da formato timestamp a calendar, in modo da poter estrarre informazioni come giorno del mese, ora del giorno oppure giorno della settimana. <br>
Si può notare che il dataset conteneva "trip_time" ossia il totale del tempo in secondi che i passeggeri hanno passato sul taxi durante il viaggio. <br>Ho preferito definire una funzione perchè sarebbe stata utile eventualmente per tutti i tempi importati dal dataset (request, pickup e dropoff).

In [4]:
import java.util.Calendar
def getDayTime(time: Long): Calendar = {
    var date:Calendar = Calendar.getInstance();
    date.setTimeInMillis(time);
    date
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import java.util.Calendar
getDayTime: (time: Long)java.util.Calendar


# Creazione e caching del RDD a partire dal file Parquet.
Caching per migliorare le prestazioni in vista di futuri utilizzi.<br>Essendoci due executor con tre core ciascuno, il numero totale di core è sei. 

In [5]:
val data_rdd = parquetFileDF.rdd
val data_rdd2 = parquetFileDF2.rdd
val rddTaxiTrip = data_rdd.map(TaxiTrip.extract)
val rddTaxiTrip2 = data_rdd2.map(TaxiTrip.extract)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data_rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[9] at rdd at <console>:28
data_rdd2: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[15] at rdd at <console>:28
rddTaxiTrip: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[16] at map at <console>:29
rddTaxiTrip2: org.apache.spark.rdd.RDD[TaxiTrip] = MapPartitionsRDD[17] at map at <console>:29


# File di input
Per tutte le seguenti query verranno utilizzati due file, i quali verranno uniti successivamente attraverso una union: il primo contenente i dati per il mese di gennaio 2021, il secondo contente i dati di febbraio 2021.
<br>Avendo caricato tutti i file nella cartella è possibile fare confronti anche su mesi diversi, ed avendo Kaggle anche i dataset di anni precedenti la stessa analisi può essere effettutata anche su anni differenti. (Attenzione ad eventuali campi mancanti in dataset di anni precedenti). 
<br>La grandezza di input è di circa 380/390 MB a file.


### Numero totale di record

In [ ]:
rddTaxiTrip.count()

### Numero totale di zone di arrivo diverse
Nota: Lo stesso numero sono quelle di partenza.

In [ ]:
rddTaxiTrip.map(x => x.endloc).distinct().count()

### Stampa delle classi di licenza 
Nel caso ne mancasse qualcuna significa che non è presente nel periodo di tempo considerato.

In [ ]:
rddTaxiTrip.map(x => x.licenseClass).distinct().collect()

###  Viaggi più lunghi in termine di KM, in relazione con la durata del viaggio

In [ ]:
val query1 = rddTaxiTrip.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))
val query1802 = rddTaxiTrip2.map(x => (x.distance, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000)))

In [ ]:
val query1union = query1.union(query1802).cache()

In [ ]:
query1union.sortByKey(false).take(10)

### Numero di viaggi effettuati da ciascun conducente in base all'ora di inizio del viaggio

In [17]:
val result = rddTaxiTrip.map(x => ((getDayTime(x.pickup.getTime()).get(Calendar.HOUR)), 1)).reduceByKey(_ + _).collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

result: Array[(Int, Int)] = Array((4,844296), (0,934327), (8,1182479), (1,886481), (9,1076083), (5,953156), (6,1082789), (10,1061788), (2,870515), (11,987859), (3,848995), (7,1179700))


### Viaggi piu costosi in termine di KM
Aggiunto un filtro sulla distanza minima. Questo comporterà solo la stampa dei viaggi più costosi visto che ci si aspetta che l'andamento sia proporzionale tra distanza e costo.

In [ ]:
val query2 = rddTaxiTrip.filter(_.distance > 20).map(x => (x.fare, ((x.dropoff.getTime() - x.pickup.getTime())/(60*1000))))
val query2802 = rddTaxiTrip2.filter(_.distance > 20).map(x => (x.fare, ((x.dropoff.getTime() - x.pickup.getTime())/(60*1000))))

In [ ]:
val query2union = query2.union(query2802).cache()

In [ ]:
query2union.sortByKey(false).take(10)

### Distanza totale percorsa da ciascuna licenza di taxi in ciascuna zona di partenza


In [ ]:
val totalDistanceByLicenseAndStartLoc = rddTaxiTrip.map(x => ((x.license, x.startloc), x.distance)).reduceByKey((x, y) => x + y).take(10)


### Tempo di viaggio medio, in base alla zona di partenza
Il codice esegue le seguenti operazioni:
* Viene eseguita una map per estrarre la zona di partenza e il tempo di viaggio;
* Successivamente viene fatta un aggregazione in base alla chiave per andare a calcolare la somma dei tempi e il conteggio dei viaggi;

Queste azioni sono state svolte per entrambi gli RDD dei mesi.

* Viene eseguita una union per raggruppare i due valori;

Il risultato finale che si sta cercando è quello di vedere per ogni zona quanto è il tempo medio di viaggio.

In [ ]:
val query3 = rddTaxiTrip.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
val query31 = rddTaxiTrip2.map(x => (x.startloc, (x.dropoff.getTime() - x.pickup.getTime())/(60*1000))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
val query3union = query3.union(query31).cache()

##### Risultato raggruppato per vedere la differenza tra i due mesi rispetto alla zona di partenza

In [ ]:
val q = query3union.map({case(k,v) => (k,v._1/v._2)}).groupByKey().map({case(k,v) => (v,k)}).sortByKey(false).take(10)

### In media quanti viaggi fa ciascuna licenza

Inizialmente, la funzione "map" viene utilizzata per estrarre le informazioni relative al numero di licenze e al numero di viaggi effettuati. Infine, la funzione "aggregate" viene utilizzata per calcolare la somma totale dei viaggi e il numero totale di licenze. Infine, viene calcolata la media.

In [ ]:
val query4 = rddTaxiTrip.map(x => (x.license,1)).
reduceByKey(_+_).
aggregate((0,0))((a,v)=>(a._1+v._2, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
"Media: " + (query4._1/query4._2)

### Numero di viaggi effettuati per ogni combinazione di ora di inizio del viaggio e zona di arrivo

In [13]:
val NTrip = rddTaxiTrip.map(x => ((getDayTime(x.pickup.getTime()).get(Calendar.HOUR), x.endloc), 1)).
reduceByKey(_ + _).
map { case ((hour, endloc), count) => (count, (hour, endloc)) }.
sortByKey(false).
collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

NTrip: Array[(Int, (Int, Long))] = Array((37314,(7,265)), (36103,(8,265)), (34085,(10,265)), (33517,(6,265)), (33365,(9,265)), (31528,(11,265)), (29066,(5,265)), (29061,(0,265)), (26894,(1,265)), (26685,(2,265)), (25943,(4,265)), (25941,(3,265)), (20991,(7,61)), (20621,(8,61)), (20286,(10,61)), (19643,(9,61)), (19287,(6,61)), (19081,(11,61)), (18057,(0,61)), (17079,(5,61)), (17069,(8,76)), (16668,(7,76)), (16421,(1,61)), (15959,(2,61)), (15618,(4,61)), (15611,(3,61)), (15520,(11,76)), (15490,(9,76)), (15273,(6,76)), (14947,(10,76)), (14935,(2,76)), (14852,(3,76)), (14849,(5,76)), (14367,(7,37)), (14302,(7,42)), (14289,(8,42)), (14129,(0,76)), (14023,(10,37)), (13843,(5,132)), (13819,(8,37)), (13782,(4,76)), (13693,(1,76)), (13310,(8,244)), (13193,(9,42)), (13146,(9,37)), (13115,(6,244))...


###  Classe di licenza che fa piu viaggi, con la presenza anche della tariffa cliente

Questo codice stampa i risultati delle classi di licenze che fa più viaggi in relazione anche al guadagno. In particolare, il codice mappa ogni elemento in una tupla in cui la chiave è la licenza, contenente il guadagno e il numero di viaggi di ogni licenza. Poi, i dati vengono aggregati tramite la funzione reduceByKey che somma i valori associati alla stessa chiave. Successivamente, il risultato viene mappato in una tupla contenente il nome della licenza, il guadagno totale e il numero di viaggi totali e viene ordinato in modo decrescente in base al guadagno. Infine, vengono stampati gli elementi della lista di risultati ottenuti dalla funzione collect().

In [ ]:
val query5 = rddTaxiTrip.map(x => (x.licenseClass,(x.fare,1))).
reduceByKey((t1,t2) => (t1._1+t2._1, t1._2+t2._2)).
map(v => (v._1,v._2._1, v._2._2)).
sortBy(_._2,false).
collect().foreach(println(_))

### Licenza che fa piu viaggi, con la presenza anche della tariffa cliente

Qui viene eseguita una replica della query precedente, basandosi su ogni singola licenza.

In [ ]:
val query6 = rddTaxiTrip.map(x => (x.license,(x.fare,1))).
reduceByKey((t1,t2) => (t1._1+t2._1, t1._2+t2._2)).
map(v => (v._1,v._2._1, v._2._2)).
sortBy(_._2,false).
collect().foreach(println(_))

### Licenza di taxi con il prezzo medio del viaggio più alto per ciascuna classe di licenza


In [ ]:
val avgFareByLicenseClass = rddTaxiTrip.map(x => (x.licenseClass, x.fare)).groupByKey().
mapValues(fares => fares.sum / fares.size).reduceByKey((fare1, fare2) => if (fare1 > fare2) fare1 else fare2)

In [ ]:
avgFareByLicenseClass.collect()

### Distanza media percorsa dai taxi in un singolo viaggio, in relazione alla zona

In [6]:
val query8 = rddTaxiTrip.filter(_.distance < 100).map(x => (x.startloc,x.distance)).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))
val query81 = rddTaxiTrip2.filter(_.distance < 100).map(x => (x.startloc,x.distance)).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query8: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[20] at aggregateByKey at <console>:29
query81: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = ShuffledRDD[23] at aggregateByKey at <console>:29


In [7]:
val query8union = query8.union(query81).cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

query8union: org.apache.spark.rdd.RDD[(Long, (Double, Double))] = PartitionerAwareUnionRDD[24] at union at <console>:29


In [8]:
val q8 = query8union.map({case(k,v) => (k,v._1/v._2)}).map({case(k,v) => (v,k)}).sortByKey(false).take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

q8: Array[(Double, Long)] = Array((20.136333333333326,1), (19.978372093023246,1), (15.583862644195618,132), (14.864652072083206,132), (11.889173913043477,2), (10.915017273061434,138), (10.765875894988065,27), (10.74933302682418,138), (10.515,199), (10.507823529411766,110))


Il codice:
* Crea una sessione Spark utilizzando SparkSession.builder.getOrCreate(). Questo garantisce che venga creata una nuova sessione Spark o che venga utilizzata una sessione esistente, se disponibile.
* Crea un array di oggetti Row utilizzando i dati ottenuti dalla query q8. Ogni oggetto Row corrisponde a una riga del DataFrame e contiene i valori delle colonne.
* Crea il DataFrame utilizzando il metodo createDataFrame di SparkSession. Viene passato l'array di righe e lo schema precedentemente definito. Il DataFrame rappresenta la tabella con le righe e le colonne corrispondenti ai dati.

In [14]:
import org.apache.spark.sql.{SparkSession, SaveMode}

// Crea la sessione Spark
val spark = SparkSession.builder.getOrCreate()

// Converte q8 in un DataFrame
val df = spark.createDataFrame(q8).toDF("Value", "Key")

// Salva il DataFrame come file CSV
df.write.format("csv").mode(SaveMode.Overwrite).option("header", "true").save("s3a://" + bucketname + "/datasets/project/output/avgDistance")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.{SparkSession, SaveMode}
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@6d251e2d
df: org.apache.spark.sql.DataFrame = [Value: double, Key: bigint]


### Zona di arrivo più comune per ciascuna licenza di taxi

In [ ]:
val mostCommonEndLocByLicense = rddTaxiTrip.map(x => ((x.license, x.endloc), 1)).reduceByKey(_ + _)
.map { case ((license, endloc), count) => (license, (endloc, count)) }.groupByKey().mapValues { locs => locs.maxBy(_._2)._1 }

In [ ]:
mostCommonEndLocByLicense.collect()

In [ ]:
import org.apache.spark.sql.SaveMode
val mostCommonEndLocDF = mostCommonEndLocByLicense.toDF("license", "endloc")
mostCommonEndLocDF.write.format("csv").mode(SaveMode.Overwrite).save("s3a://"+bucketname+"/datasets/project/output/MaxLoc")

### Analisi sul guadagno medio di ogni singolo viaggio per ogni giorno
Il codice ha lo scopo di calcolare il guadagno medio di ogni classe di licenza in base al giorno in cui si è svolto il viaggio. Il risultato è stato poi salvato in un file CSV.<br>
Sono state eseguite le seguenti operazioni:
* Viene eseguito creato un RDD (avgFareByDay) che associa a ciascuna coppia (classe di licenza, Giorno) una tupla contenente la somma del guadagno e il numero di viaggi durante il giorno.
* Viene quindi eseguita una riduzione per chiave (reduceByKey) per aggregare i dati raggruppati per classe di licenza e giorno.
* Successivamente, viene eseguita una mappatura (mapValues) per calcolare il guadagno medio di ogni viaggio per ogni giorno.
* In seguito, viene eseguita una mappatura (map) per riorganizzare i dati e raggrupparli per classe di licenza.
* Vengono poi eseguite ulteriori operazioni di trasformazione (flatMap e coalesce) per preparare i dati per il salvataggio su file CSV.
* Infine, viene convertito l'RDD in un DataFrame e salvato in un file CSV.


In [28]:
val avgFareByDay = rddTaxiTrip.map(x => ((x.licenseClass, getDayTime(x.pickup.getTime()).get(Calendar.DAY_OF_WEEK)), (x.fare, 1))).
  reduceByKey((accum, value) => (accum._1 + value._1, accum._2 + value._2)).
  mapValues(sumCount => sumCount._1 / sumCount._2).
  map(item => (item._1._1, item._2, item._1._2))
  //collect() //(licenza,avg,giorno)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

avgFareByDay: org.apache.spark.rdd.RDD[(String, Double, Int)] = MapPartitionsRDD[29] at map at <console>:51


In [29]:
// Raggruppo per licenza
val avgFareByLicense = avgFareByDay.groupBy(_._1).
  mapValues(_.map(item => (item._2, item._3)))//.collect()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

avgFareByLicense: org.apache.spark.rdd.RDD[(String, Iterable[(Double, Int)])] = MapPartitionsRDD[32] at mapValues at <console>:49


In [30]:
//flatmap con chiave prodotto e anno
val avgFareByLicenseAndDay = avgFareByLicense.flatMap { 
    case (license, dayFare) => dayFare.map { 
        case (avgFare, day) => ((license, day),avgFare)
  }
}.coalesce(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

avgFareByLicenseAndDay: org.apache.spark.rdd.RDD[((String, Int), Double)] = CoalescedRDD[34] at coalesce at <console>:52


In [31]:
import org.apache.spark.sql.SaveMode
// Conversione in DataFrame con stringa
val df = avgFareByLicenseAndDay.map{case ((license, day), avgFare) => (license,day,avgFare)}.toDF()

// Salvataggio su file CSV
df.write.format("csv").mode(SaveMode.Overwrite).save("s3a://"+bucketname+"/datasets/project/output/avgFare")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SaveMode
df: org.apache.spark.sql.DataFrame = [_1: string, _2: int ... 1 more field]
